### On the unit commitment problem
This notebook contains the dynamic programming solution to a 2-unit commitment problem.

The notation, state transition dynamics and cost, follow that in:

``Lauer, G.S., Sandell, N.R., Bertsekas, D.P. and Posbergh, T.A., 1982. Solution of large-scale optimal unit commitment problems. IEEE Transactions on Power Apparatus and Systems, (1), pp.79-86.''

In [1]:
# Loading used packages
using Plots, LaTeXStrings

We first define the state transition dynamics, as defined in Fig. 1 in the above reference.

In [ ]:
function state_transition(state::Int, action::Int)
    stateVec = zeros(Int, 11,)
    stateVec[state] = 1
    # We define the state transition matrix P
    P = zeros(Int, 11, 11)
    if action == 0
    P[[
    CartesianIndex(1,4), 
    CartesianIndex(2,1),    CartesianIndex(3,2),    CartesianIndex(4,5),    
    CartesianIndex(5,6),    CartesianIndex(6,7),    CartesianIndex(7,8),
    CartesianIndex(8,9),    CartesianIndex(9,10),
    CartesianIndex(10,11),    CartesianIndex(11,11)]] .= 1

    elseif action >= 1
    P[[
    CartesianIndex(1,1),    CartesianIndex(2,1),    CartesianIndex(3,2),
    CartesianIndex(4,5),    CartesianIndex(5,6),    CartesianIndex(6,7),
    CartesianIndex(7,3),    CartesianIndex(8,3),    CartesianIndex(9,3),
    CartesianIndex(10,3),    CartesianIndex(11,3)]] .= 1
    end
    return findfirst(==(1), P' * stateVec)

end

We have 11 states according to the figure, the states {1,2,3} correspond to the up states, and {4,...,11} are the down states. The states are ordered top to bottom as in Fig. 1.

The control action takes the value 0 when shutting down, and the values {1,...,GridSize} when starting up.

The reason of multiple values of action for the starting up is to determine the output power of the unit while running. The value 1 corresponds to the unit being running but with the lowest output power $\underline g_i$ (g_min in code) and GridSize to the maximum output $\bar g_i$ (g_max in code). G_grid is the number of grid points when we discretize the output power $g_i^t \in [\underline g_i,\bar g_i]$.

In [4]:
GridSize = 20
g_min = 1
g_max = 2
OutPowerGrid = [0; range(g_min, g_max, GridSize)];

Notice that in the state_transition function, action that is in {1,...,GridSize} resulted in the same transition (u=1 in the paper). This is because the 11 states are only affected by the {on, off} decision, but not affected by the output power $g_i^t$ when the unit $i$ is on. Therefore, since the output power has no dynamics (as long as the unit is on), we regard it as a constrained input rather than a state.

The function below describes the set of feasible actions $A$ for each state $s$.

In [6]:
function feasible_action_set(s::Int)
    if s in [1,7,8,9,10,11]
        A = 0:GridSize
    elseif s in [2,3]
        A = 1:GridSize
    elseif s in [4,5,6]
        A = 0
    end
    return A
end;

That is, the feasible action set $A(s)$ is a function of the state $s$:
- the feasible action set can only contain $0$ for states $\{4,5,6\}$ (minimum down time not achieved yet; the unit cannot be truned up)
- the feasible action set can be all possible actions $\{0,...,GridSize\}$ for states $\{1,7,8,...,11\}$.
- the feasibile action set is $\{1,...,GridSize\}$ for states 2 and 3 (minimum up time not achieved yet).